# 梯度下降
梯度下降是理解梯度下降算法的关键

预处理 preconditioning 是梯度下降中的一种常用技术，还被沿用到更高级的算法中。



In [ ]:
# f(x+ε)= f(x)+ εf'(x) + O(ε**2)
# 在一阶近似中，f(x+ε)可以通过在x处的函数值和一阶导数得出，
# 假设在负梯度方向上移动的ε会减少f

%matplotlib inline
import numpy as np
import torch as t 
from pltutils import *

def f(x):
    return  x**2
def f_grad(x):
    return 2*x

def gd(eta,f_grad):
    pass
